# MuseGAN
[Hao-Wen Dong](https://salu133445.github.io/), [Wen-Yi Hsiao](https://github.com/wayne391), [ Li-Chia Yang](https://richardyang40148.github.io/), [Yi-Hsuan Yang](http://mac.citi.sinica.edu.tw/~yang/)

MuseGAN — это проект по генерации музыки. Цель авторов этого проекта в создании полифонической (с равноправием отдельных "треков" (инструментов) в мультитрековой музыке) музыки из нескольких треков (инструментов). 

Предоставленные модели могут генерировать музыку, как с нуля, так и с трека данного пользователем. Нам понадобится второй способ.

Модель натренирована с данных, собранных из [Lakh Pianoroll Dataset](https://salu133445.github.io/lakh-pianoroll-dataset/) чтобы генерировать куски популярных песен, состоящих из треков баса, ударных, гитары, пианино и струнных.

## Аннотация

Генерация музыки имеет несколько значимых отличий от генерации картинок и видео. Во-первых, музыка - это искусство времени, потому требуется временн**а**я модель (temporal model). Во-вторых, музыка обычно исполняется несколькими инструментами/треками с их собственной временной динамикой, но вместе они взаимозависимо раскрываются со временем. В-третьих, ноты чаще всего сгруппирированы в аккорды, арпеджио (последовательно идущие звуки аккорда) или мелодии в полифонической музыке, и тем самым введение хронологического порядка нот не подходит. Предлагаются три GAN модели (Generative Adversarial Networks - "Генеративно-состязательные сети") для символьной генерации мультитрековой музыки. Эти модели, отличающиеся друг от друга исходными предположениями и, соответственно, архитектурами сетей, называются импровизирующая модель ("джемящая", jamming model), композиторская модель (the composer model) и гибридная модель (the hybrid model). Вышеупомянутые модели были натренированы на датасете, состоящем из более ста тысяч тактов (кусков) из рок-музыки, применённых для генерации пианороллов (piano roll) в виде пяти треков: баса, ударных, гитары, пианино и струнных. В дополнение к субъективной человеческой оценке, несколько интра-трековых и интер-трековых объективных метрик также были добавлены для оценки результатов генерации. Данные модели могут генерировать связную музыку из четырёх тактов прямо с нуля (т.е. без созданных человеком входных данных). Также модели были расширены для генерации музыки "руками" человека и ИИ: получив трек, созданный человеком, модель может сгенерировать четыре дополнительных трека для аккомпонемента.

## Модель

Модель MuseGAN состоит из двух частей: мультитрековая модель (multitrack model) и временн**а**я модель (temporal model). Мультитрековая модель ответственна за взаимозависимость треков, в то время как временная модель справляется с временной зависимостью. Предоставлены три мультитрековых модели, в соотвествии с тремя распространёнными композиционными подходами. Для временной модели, предоставлены модель для генерации с нуля, и другая для аккомпанирования треку, заранее данной пользователем.

### Моделирование взаимозависимости треков в мультитреке

По опыту авторов, есть два общепринятых способа создавать музыку:

- Когда группа музыкантов играет на различных инструментах, они могут создавать музыку импровизируя без предопределённой аранжировки, т.е. джемминг.
- Композитор аранжирует инструменты, зная гармоничную структуру и инструментровку. Музыканты будут играть музыку, следя за композицией.

Три модели были созданы по соотношению к этим трём подходам к созданию музыки:

#### Импровизирующая (джемящая) модель

Несколько генераторов работают независимо и генерируют музыку их собственного трека из личного случайного вектора $z_i$ ($i = 1..M)$, где $M$ означает количество генераторов (или треков). Эти генераторы получают критику (т.е. обратносвязные надзорные сигналы) от различных дискриминаторов.

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/multitrack1.png" width="400px">
</p>

#### Композиторская модель

Один генератор создаёт мультиканальный пианоролл (pianoroll), с каждым каналом представляющим определённый трек. Эта модель требует только один общий случайный вектор $z$ (на который можно смотреть как на замысел композитора) и один дискриминатор, который изучает все $M$ треков чтобы сказать, реальна ли вводная музыка. 

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/multitrack2.png" width="400px">
</p>

#### Гибридная модель

Комбинируя идеи джемминга и композиции, гибридная модель требует $M$ генераторов и каждый берёт на вход интер-трековый случайный вектор $z$ и интра-трековый случайный вектор $z_i$. Ожидаемо что интер-трековый случайный вектор может координировать генерацию различных музыкантов, названных $G_i$, прямо как это делает композитор. Вдобавок, используется только один дискриминатор для оценки всех $M$ треков.

> Основное отличие между композиторской моделью и гибридной моделью лежит в гибкости — в гибридной модели можно использовать различные архитектуры сетей (например, числов слоёв, размер фильтра) и различные входные данные для M генераторов. Следовательно, можно например варьировать генерацию одного определённого трека без потери интер-трековой взаимозависимости.

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/multitrack3.png" width="400px">
</p>

### Моделирование временн**о**й структуры

#### Генерация с нуля

Первый метод заключается в генерации музыкальных фраз фиксированной длины путём взгляда на последовательность такта как на другое измерение для роста генератора. Генератор состоит из двух подсетей, *генератора временной структуры $G_{temp}$ (temporal structure generator)* и *тактовый генератор $G_{bar}$ (bar generator)*. $G_{temp}$ сопоставляет вектор шума к последовательности некоторых скрытых (latent) векторов, в которых должна хранится информация о времени и использована $G_{bar}$'ом для генерации пианороллов последовательно (т.е. такт за тактом).

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/temporal1.png" width="600px">
</p>

#### Генерация на основе трека

Второй метод основывается на том что такт одного определённого трека дан человеком и модель пытается выучить временн**у**ю структуру лежащую в основе того трека и сгенерировать оставшиеся треки (и завершить мелодию). *Основывающийся на треке генератор $G°$ (track-conditional generator)* генирирует такты один за другим с *условным тактовым генератором, $G°_{bar}$ (conditional bar generator)*, который берёт на вход основывающий трек и случайный шум. Для того чтоб достигнуть такую условную генерацию с высоко-размерными условиями, дополнительный энкодер $E$ натренирован чтобы сопоставлять условие к пространству $z$.
>Учтите что энкодер должен извлекать интер-трековые признаки (features), вместо интра-трековых, из поданного трека, поскольку интра-трековые признаки не будут полезны для генерирования других треков.

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/temporal2.png" width="600px">
</p>

### MuseGAN

MuseGAN, интеграция и расширение предоставленных мультитрековой и временной моделей, берёт на вход четыре разных типа случайных векторов:
- интер-трековый, независимый от времени случайный вектор $z$;
- интер-трековый, зависимый от времени случайный вектор $z_t$;
- $M$ интра-трековых, независимых от времени случайных векторов $z_i$;
- $M$ интра-трековых, зависимых от времени случайных векторов $z_{i,t}$.

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/musegan-generator.png" width="600px">
</p>

Для $i$-ого трека ($i=1..M$), *общий* генератор временных структур $G_{temp}$ и *приватный* генератор временных структур $G_{temp, i}$ берут зависимые от времени случайные вектора $z_t$ и $z_{i,t}$ соответственно, как и их входные данные, и каждый из них выводит ряд скрытых векторов содержащих интер-трековую и интра-трековую соответственно временную информацию. Выходящий ряд (скрытых векторов) вместе с независимыми от времени случайными векторами $z$ и $z_i$ объединяются и подаются тактовому генератору $G_{bar}$ который потом генерирует пианороллы последовательно.

Ниже представлена диаграмма всей системы.

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/musegan.png" width="800px">
</p>

## Данные

### Lakh Pianoroll Dataset

В данном проекте используется *подчищенная* (со слов авторов) версия датасета [Lakh Pianoroll Dataset](https://salu133445.github.io/lakh-pianoroll-dataset/) (LPD). LPD содержит 174154 уникальных [мультитрековых пианороллов](https://salu133445.github.io/lakh-pianoroll-dataset/representation) (pianoroll), собранных из MIDI-файлов из [Lakh MIDI Dataset](https://colinraffel.com/projects/lmd/) (LMD), когда же подчищенная версия содержит 21425 пианороллов, которые в размере 4/4 и были сопоставлены к определённым записям в [Million Song Dataset](https://labrosa.ee.columbia.edu/millionsong/) (MSD).

### Данные для обучения

- Используется *тайминг символов*, который отбрасывает информацию о темпе (больше информации [тут](https://salu133445.github.io/lakh-pianoroll-dataset/representation);
- Отбрасывается информация о скорости (поскольку используются основанные на двоичных значениях пианороллы);
- 84 возможностей для высоты звука (от "До" первой октавы (C1) до "Си" седьмой (B7));
- Треки объединены в пять категорий: Бас (Bass), Ударные (Drums), Гитара (Guitar), Пианино (Piano) и Струнные (Strings);
- Берутся мелодии лишь с тэгом "рок" (`rock`)
- Собраны музыкально имеющие смысл 4-тактовые фразы для временной модели, с делением пианороллов `структурными особенностями (features)`, предложенными в [1].

В результате, размер выходного тензора — 4 (такты) × 96 (временные шаги) × 84 (высота (тональность)) × 5 (треков (инструментов)).

Ниже представлены два примера пианороллов, которые есть в данных для обучения MuseGAN. Треки (сверху вниз): Бас, Ударные, Гитара, Струнные, Пианино.

<p align="center">
    <img src="https://salu133445.github.io/musegan/figs/train_samples.png" width="600px">
</p>

### Источники

1. Joan Serrá, Meinard Müller, Peter Grosche and Josep Ll. Arcos, “Unsupervised Detection of Music Boundaries by Time Series Structure Features,” in AAAI Conference on Artificial Intelligence (AAAI), 2012


---
## Работа с MuseGAN

MuseGAN требует Tensorflow 1.x версии. В Google Colab, к счастью, есть возможность переключиться на первую версию (при помощи `%tensorflow_version 1.x`).

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

Также тут есть большая часть необходимых библиотек, кроме
```
absl-py
grpcio
pypianoroll==0.5.3
```
<font color="gray">
(и других, но я не помню каких).
</font>



Для начала, скачаем данные и модель.

In [3]:
%cd ./drive/MyDrive/musegan/musegan-main

/content/drive/MyDrive/musegan/musegan-main


In [4]:
 !./scripts/download_data.sh

Downloading...
From: https://drive.google.com/uc?id=14rrC5bSQkB9VYWrvt2IhsCjOKYrguk3S
To: /content/drive/MyDrive/musegan/musegan-main/data/train_x_lpd_5_phr.npz
103MB [00:01, 91.2MB/s] 


In [5]:
!./scripts/download_models.sh

Downloading...
From: https://drive.google.com/uc?id=19RYAbj_utCDMpU7PurkjsH4e_Vy8H-Uy
To: /content/drive/MyDrive/musegan/musegan-main/exp/pretrained_models.tar.gz
429MB [00:03, 110MB/s]
Decompressing archive.
Successfully decompressed.


Есть два скрипта для получения результатов из предобученной модели:
- `run_inference.sh`
- `run_interpolation.sh`

По словам одного из авторов, первый, inference script, генерирует несколько образцов из входных шумов, взятых из скрытого пространства (который использовался при обучении). Потому получаются буквально случайные образцы. (Изображения предоставлены одним из авторов)

<img src="https://user-images.githubusercontent.com/22392791/59755147-62c68980-92c2-11e9-87e7-3b98dfb1d3b2.png">

Второй же, interpolation script, генерирует образцы так же, только входные шумы взяты из скрытого пространства уже как сеть (grid). Таким образом можно наблюдать как сгенерированые образцы меняются с изменением входных данных.

<img src="https://user-images.githubusercontent.com/22392791/59755092-4e828c80-92c2-11e9-8dd1-d6cf28d0ddbd.png">

Запустим оба и взглянем на результаты.

In [10]:
!./scripts/run_inference.sh "./exp/default/" "0"

musegan.inference    INFO     Using parameters:
{'beat_resolution': 12,
 'condition_track_idx': None,
 'data_shape': [4, 48, 84, 5],
 'is_accompaniment': False,
 'is_conditional': False,
 'latent_dim': 128,
 'nets': {'discriminator': 'default', 'generator': 'default'},
 'use_binary_neurons': False}
musegan.inference    INFO     Using configurations:
{'adam': {'beta1': 0.5, 'beta2': 0.9},
 'batch_size': 64,
 'checkpoint_dir': './exp/default//model',
 'colormap': [[1.0, 0.0, 0.0],
              [1.0, 0.5, 0.0],
              [0.0, 1.0, 0.0],
              [0.0, 0.0, 1.0],
              [0.0, 0.5, 1.0]],
 'columns': 5,
 'config': './exp/default//config.yaml',
 'data_filename': 'train_x_lpd_5_phr',
 'data_root': None,
 'data_source': 'sa',
 'evaluate_steps': 100,
 'gan_loss_type': 'wasserstein',
 'gpu': '0',
 'initial_learning_rate': 0.001,
 'learning_rate_schedule': {'end': 50000, 'end_value': 0.0, 'start': 45000},
 'log_loss_steps': 100,
 'lower': -2,
 'midi': {'is_drums': [1, 0, 0, 0, 0

In [11]:
!./scripts/run_interpolation.sh "./exp/default/" "0"

musegan.interpolation INFO     Using parameters:
{'beat_resolution': 12,
 'condition_track_idx': None,
 'data_shape': [4, 48, 84, 5],
 'is_accompaniment': False,
 'is_conditional': False,
 'latent_dim': 128,
 'nets': {'discriminator': 'default', 'generator': 'default'},
 'use_binary_neurons': False}
musegan.interpolation INFO     Using configurations:
{'adam': {'beta1': 0.5, 'beta2': 0.9},
 'batch_size': 64,
 'checkpoint_dir': './exp/default//model',
 'colormap': [[1.0, 0.0, 0.0],
              [1.0, 0.5, 0.0],
              [0.0, 1.0, 0.0],
              [0.0, 0.0, 1.0],
              [0.0, 0.5, 1.0]],
 'columns': 5,
 'config': './exp/default//config.yaml',
 'data_filename': 'train_x_lpd_5_phr',
 'data_root': None,
 'data_source': 'sa',
 'evaluate_steps': 100,
 'gan_loss_type': 'wasserstein',
 'gpu': '0',
 'initial_learning_rate': 0.001,
 'learning_rate_schedule': {'end': 50000, 'end_value': 0.0, 'start': 45000},
 'log_loss_steps': 100,
 'lower': 0.0,
 'midi': {'is_drums': [1, 0, 0, 0

Полученные результаты находятся в `./exp/default/results`, в папках `inference` и `interpolation` соответственно. В каждой папке находятся три папки с тремя разными видами тех же результатов:
- `arrays` с файлами `.npy` - массивами numpy;
- `images` с файлами `.png` - изображениями пианороллов;
- `pianorolls` с файлами `.npz` - мультитрековыми пианороллами, которые можно конвертировать в MIDI-файлы с помощью `pypianoroll`.

Сконвертируем все "`.npz`" в "`.mid`" рядом с оригинальными файлами.

In [14]:
from pypianoroll import Multitrack
import os
for subdir, dirs, files in os.walk("./exp/default/results/"):
    for f in files:
        if f.endswith(".npz"):
            m = Multitrack(os.path.join(subdir, f))
            m.write((os.path.join(subdir, f))[:-3]+"mid")

Результаты выложил на [свой GitHub](https://github.com/littlebomb/musegan-results). 

Пример инференсного результата в виде изоображения:

<img src="https://github.com/littlebomb/musegan-results/raw/main/inference/images/fake_x_colored/fake_x_colored_0.png">

И интерполяционного:

<img src="https://github.com/littlebomb/musegan-results/raw/main/interpolation/images/fake_x_colored/fake_x_colored_0.png">

---

## Литература
1. MuseGAN [Электронный ресурс]. URL: https://salu133445.github.io/musegan (дата обращения 28.07.2021).
2. salu133445/musegan: An AI for Music Generation [Электронный ресурс]. URL: https://github.com/salu133445/musegan (дата обращения 28.07.2021).